<a href="https://colab.research.google.com/github/BehnoodRasti/Unmixing_Tutorial_IEEE_IADF/blob/main/SMALU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sparse (Semisupervised) Unmixing - SMALU


---

original implementation found at https://github.com/YuanhangLin/Whispers2022Unmixing

Published in

Y. Lin and P Gader, "ADDRESSING SPECTRAL VARIABILITY IN HYPERSPECTRAL UNMIXING WITH UNSUPERVISED NEURAL NETWORKS," in IEEE WHISPERS.

## Imports

1. Clone required github repositories
2. Import Python packages

In [8]:
!git clone https://github.com/BehnoodRasti/SUnCNN.git

fatal: destination path 'SUnCNN' already exists and is not an empty directory.


In [9]:
from scipy.io import loadmat
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import pearsonr
from sklearn.preprocessing import normalize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pdb
import numpy as np
import copy
import random

In [10]:
import scipy.io
import numpy as np
fname2  = "SUnCNN/Data/DC2/Y_clean.mat"
mat2 = scipy.io.loadmat(fname2)
img_np_gt = mat2["Y_clean"]
img_np_gt = img_np_gt.transpose(2,0,1)
[p1, nr1, nc1] = img_np_gt.shape
img_resh=np.reshape(img_np_gt,(p1,nr1*nc1))
#%%
fname3  = "SUnCNN/Data/DC2/XT.mat"
mat3 = scipy.io.loadmat(fname3)
A_true_np = mat3["XT"]

#%%
fname4  = "SUnCNN/Data/DC2/EE.mat"
mat4 = scipy.io.loadmat(fname4)
EE = mat4["EE"]
#%%
LibS=EE.shape[1]

In [11]:
LibS

240

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

!nvidia-smi

cpu
NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [13]:
#library = loadmat("/content/drive/MyDrive/ColabData/MUA-SparseUnmixing/spectral_library_urban.mat", squeeze_me = True)
#label_names = library["material_names"].astype(str)
#X = loadmat("/content/drive/MyDrive/ColabData/MUA-SparseUnmixing/Urban_R162.mat", squeeze_me = True)
#wl, X, H, W = np.linspace(400, 2500, 224)[X["SlectBands"]], X["Y"].astype(float) / X["maxValue"], X["nRow"], X["nCol"]
#X = X.T
dtype = torch.cuda.FloatTensor
X=img_resh.T

#library.keys(), library["A"].shape, X.shape

In [ ]:
X = torch.Tensor(X).type(dtype).to(device)
X.requires_grad = False

E = torch.FloatTensor(EE.T).type(dtype)
E = E.to(device)
E.requires_grad = False

In [ ]:
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

def BrayCurtisSimilarity(X, X_hat):
    return (torch.abs(X - X_hat).sum(dim = 1) / (X.sum(dim = 1) + X_hat.sum(dim = 1))).mean()


def SID(X, X_hat):
    p = (X / X.sum(dim = 1, keepdim = True)) + 1e-3
    q = (X_hat / X_hat.sum(dim = 1, keepdim = True)) + 1e-3
    return torch.sum(p * torch.log(p / q) + q * torch.log(q / p), dim = 1).mean()

def COS(X, X_hat):
    X_norm = X.norm(p = 2, dim = 1)
    X_hat_norm = X_hat.norm(p = 2, dim = 1)
    return 1 - (X * X_hat).sum(dim = 1) / (X_norm * X_hat_norm)

def SAD(X, X_hat):
    X_norm = X.norm(p = 2, dim = 1)
    X_hat_norm = X_hat.norm(p = 2, dim = 1)
    inner_product = (X * X_hat).sum(dim = 1)
    return torch.acos(inner_product / (X_norm * X_hat_norm)).mean()

def SIDSAM(X, X_hat):
    X_norm = X.norm(p = 2, dim = 1)
    X_hat_norm = X_hat.norm(p = 2, dim = 1)
    inner_product = (X * X_hat).sum(dim = 1)
    alpha = torch.acos(inner_product / (X_norm * X_hat_norm))

    p = (X / X.sum(dim = 1, keepdim = True))
    q = (X_hat / X_hat.sum(dim = 1, keepdim = True))
    sid =  torch.sum(p * torch.log(p / q + 1) + q * torch.log(q / p + 1), dim = 1)

    return (sid * torch.tan(alpha)).mean()

def JMSAM(X, X_hat):
    m_t, m_r = X.mean(dim = 1), X_hat.mean(dim = 1)
    sigma_t, sigma_r = X.var(dim = 1), X_hat.var(dim = 1)
    sigma_avg = (sigma_t + sigma_r) / 2
    B = 1/8 * (m_t - m_r)**2 / sigma_avg + 1/2*torch.log(torch.abs(sigma_avg) / torch.sqrt(sigma_t * sigma_r))
    X_norm, X_hat_norm = X.norm(p = 2, dim = 1), X_hat.norm(p = 2, dim = 1)
    inner_product = (X * X_hat).sum(dim = 1)
    alpha = torch.arccos(inner_product / (X_norm * X_hat_norm))
    return (2*(1-torch.exp(-B))*torch.tan(alpha)).mean()

def NS3(X, X_hat):
    _, B = X.shape
    l2_dist = torch.sqrt(((X - X_hat) ** 2).sum(dim = 1)/B)
    X_norm = X.norm(p = 2, dim = 1)
    X_hat_norm = X_hat.norm(p = 2, dim = 1)
    inner_product = (X * X_hat).sum(dim = 1)
    cos_alpha = inner_product / (X_norm * X_hat_norm)

    lower, upper = torch.min(X, dim = 1)[0], torch.max(X, dim = 1)[0]
    l2_dist = (l2_dist - lower) / (upper - lower)
    return torch.sqrt(l2_dist**2 + (1 - cos_alpha)**2).mean()

class Sparsemax(nn.Module):
    """Sparsemax function."""

    def __init__(self, dim=None):
        """Initialize sparsemax activation
        
        Args:
            dim (int, optional): The dimension over which to apply the sparsemax function.
        """
        super(Sparsemax, self).__init__()
        self.dim = -1 if dim is None else dim

    def forward(self, input):
        """Forward function.
        Args:
            input (torch.Tensor): Input tensor. First dimension should be the batch size
        Returns:
            torch.Tensor: [batch_size x number_of_logits] Output tensor
        """
        # Sparsemax currently only handles 2-dim tensors,
        # so we reshape to a convenient shape and reshape back after sparsemax
        input = input.transpose(0, self.dim)
        original_size = input.size()
        input = input.reshape(input.size(0), -1)
        input = input.transpose(0, 1)
        dim = 1

        number_of_logits = input.size(dim)

        # Translate input by max for numerical stability
        input = input - torch.max(input, dim=dim, keepdim=True)[0].expand_as(input)

        # Sort input in descending order.
        # (NOTE: Can be replaced with linear time selection method described here:
        # http://stanford.edu/~jduchi/projects/DuchiShSiCh08.html)
        zs = torch.sort(input=input, dim=dim, descending=True)[0]
        range = torch.arange(start=1, end=number_of_logits + 1, step=1, device=device, dtype=input.dtype).view(1, -1)
        range = range.expand_as(zs)

        # Determine sparsity of projection
        bound = 1 + range * zs
        cumulative_sum_zs = torch.cumsum(zs, dim)
        is_gt = torch.gt(bound, cumulative_sum_zs).type(input.type())
        k = torch.max(is_gt * range, dim, keepdim=True)[0]

        # Compute threshold function
        zs_sparse = is_gt * zs

        # Compute taus
        taus = (torch.sum(zs_sparse, dim, keepdim=True) - 1) / k
        taus = taus.expand_as(input)

        # Sparsemax
        self.output = torch.max(torch.zeros_like(input), input - taus)

        # Reshape back to original shape
        output = self.output
        output = output.transpose(0, 1)
        output = output.reshape(original_size)
        output = output.transpose(0, self.dim)

        return output

    def backward(self, grad_output):
        """Backward function."""
        dim = 1

        nonzeros = torch.ne(self.output, 0)
        sum = torch.sum(grad_output * nonzeros, dim=dim) / torch.sum(nonzeros, dim=dim)
        self.grad_input = nonzeros * (grad_output - sum.expand_as(grad_output))

        return self.grad_input

class ParingLayer(nn.Module):
    def __init__(self, n_features):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(n_features, n_features))

    def forward(self, H, V):
        return H @ self.weight @ V.T


def plot_all_endmembers(x, a, wl, E, labels, label_names):
    if isinstance(a, torch.Tensor):
        a = a.detach().cpu().numpy()
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    if isinstance(E, torch.Tensor):
        E = E.detach().cpu().numpy()
    indices = np.where(a != 0)[0]

    plt.subplot(1, 2, 1)
    plt.plot(wl, x.T, c = "b", label = "real")
    plt.plot(wl, (a @ E).T, c = "r", label = "reconstructed")
    plt.ylim([0, 1])
    plt.legend()

    plt.subplot(1, 2, 2)
    for idx in indices:
        plt.plot(wl, E[idx].T, alpha = a[idx] * 10, 
                 label = label_names[labels[idx]] + "#" + str(idx) + " " + str(np.round(a[idx], 2)))
    plt.legend()
    plt.ylim([0, 1])
    plt.show()
    
def plot_all_endmembers_merges(x, a, wl, E, labels, label_names):
    if isinstance(a, torch.Tensor):
        a = a.detach().cpu().numpy()
    if isinstance(x, torch.Tensor):
        x = x.detach().cpu().numpy()
    if isinstance(E, torch.Tensor):
        E = E.detach().cpu().numpy()
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()

    plt.subplot(1, 2, 1)
    plt.plot(wl, x.T, c = "b", label = "real")
    plt.plot(wl, (a @ E).T, c = "r", label = "reconstructed")
    plt.ylim([0, 1])
    plt.legend()

    plt.subplot(1, 2, 2)
    for c in range(6):
        flag = (a > 0) & (labels == c)
        if np.any(flag):
            weighted_mean = np.average(E[flag], weights=a[flag], axis = 0)
            plt.plot(wl, weighted_mean.T, label = label_names[c] + " " + str(np.round(a[flag].sum(), 2)))
            # print(label_names[c], np.round(a[flag].sum(), 2))
    plt.legend()
    plt.ylim([0, 1])
    plt.show()
    

# plot_all_endmembers(x[0], A[0], wl[bbl_flag == 1], E, labels_full, label_names)
# plot_all_endmembers_merges(x[0], A[0], wl[bbl_flag == 1], E, labels_full, label_names)

In [ ]:
class MyMLP(nn.Module):

    class Embedding(nn.Module):
        def __init__(self, n_features, library_size):
            super().__init__()
            self.weight = nn.Parameter(torch.rand(n_features, library_size))

        def forward(self, H):
            return H @ self.weight

    def __init__(self, B, library_size):
        super(MyMLP, self).__init__()

        self.fc1 = nn.Linear(B, B//2)
        self.bn1 = nn.BatchNorm1d(B//2)
        self.fc2 = nn.Linear(B//2, B//4)
        self.bn2 = nn.BatchNorm1d(B//4)
        self.fc3 = nn.Linear(B//4, B//8)
        self.bn3 = nn.BatchNorm1d(B//8)

        self.spectra_encoder = [(self.fc1, self.bn1, nn.ReLU()), 
                                (self.fc2, self.bn2, nn.ReLU()),
                                (self.fc3, self.bn3, nn.Tanh())]

        self.embeddings = nn.Linear(B//8, library_size, bias = False)

        for fc in [self.fc1, self.fc2, self.fc3, self.embeddings]:
            torch.nn.init.xavier_normal_(fc.weight)
        
    def encoding_spectra(self, X):
        H = X
        for fc, bn, activation in self.spectra_encoder:
            H = fc(H)
            if bn is not None: H = bn(H)
            if activation is not None: H = activation(H)
        return H

    def forward(self, X):
        H = self.encoding_spectra(X)
        inner_product = self.embeddings(H)
        A = Sparsemax(dim = 1)(inner_product)
        return A


from collections import Counter
#ctr = Counter(labels_full.detach().cpu().numpy())
#foo = [ctr[key] for key in range(6)]

endmember_ranges, acc = [], 0
#for i in range(6):
#    endmember_ranges.append(np.arange(acc, acc + foo[i]))
#    acc += foo[i]

#def merge_full_to_6(A, endmember_ranges):
#    A_merged = np.zeros((len(A), 6))
#    for i in range(len(A)):
#        row = A[i]
#        for j in range(len(endmember_ranges)):
#            A_merged[i, j] = np.sum(row[endmember_ranges[j]])
#    return A_merged

#def merge_6_to_3(A):
#    A_merged = np.zeros((len(A), 3))
#    gv_indices = np.array([0, 5])
#    pervious_indices = np.array([1, 4])
#    impervious_indices = np.array([2, 3])
#    A_merged[:, 0] = np.sum(A[:, gv_indices], axis = 1)
#    A_merged[:, 1] = np.sum(A[:, pervious_indices], axis = 1)
#    A_merged[:, 2] = np.sum(A[:, impervious_indices], axis = 1)
#    return A_merged



import time
cleanup, verbose = True, False
cleanup_threshold = 1e-2 
lr, individual_maes, merged_maes = 10e-3, [], []

for seed in range(1):
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)
    model = MyMLP(p1, len(E)).to(device)
    start_time = time.time()
    optimizer = torch.optim.AdamW(model.parameters(), lr = lr, weight_decay = 1e-2)

    for epoch in range(500):
        model.train()
        A = model(X)
        
        if epoch >= 200 and cleanup:
            mask = torch.ones(A.shape).to(device) 
            mask[A <= cleanup_threshold] = 0
            A *= mask
            A /= A.sum(dim = 1, keepdim = True)

        X_hat = A @ E

        optimizer.zero_grad()
        total_loss = nn.MSELoss(reduction = "mean")(X_hat, X) 
        total_loss.backward()
        optimizer.step()

        print("epoch %d, loss: %.7f" % (epoch, total_loss.detach().item()))
    print("\nexecution time: %.3f, seed: %d," %(time.time() - start_time, seed), "cleanup?", cleanup)
    print("arch:", model.spectra_encoder, "\noptimizer:", optimizer)

A_est=A.T.detach().cpu().numpy()
#    A_est = merge_full_to_6(A.detach().cpu().numpy(), endmember_ranges)
#    fig, axs = plt.subplots(nrows=1, ncols=6, figsize = [20, 3])
#    for c in range(6):
#        axs[c].imshow((A_est[:, c].reshape((H, W)).T), aspect="auto")
#        axs[c].set_title("EST " + label_names[c])
#    plt.show()

In [ ]:
from numpy import linalg as LA
A_est=A.T.detach().cpu().numpy()
A_true=A_true_np.astype(np.float32).reshape((EE.shape[1],nr1*nc1))

SRE=10*np.log10(LA.norm(A_true,'fro')/LA.norm((A_true.astype(np.float32)- np.clip(A_est, 0, 1)),'fro'))
print ('SRE: %f' % (SRE))

In [ ]:
plt.plot(A_est.reshape(LibS,nr1*nc1))
